# Coulomb gauge transformation

### Simulation parameters

In [1]:
import numpy as np

# hbar * c [GeV * fm]
hbarc = 0.197326 

# Simulation box 
L = 2      
N = 64   
tau_sim = 0.1     
DTS = 4     

# Glasma fields
su_group = 'su3'
Qs = 2        
ns = 50    
factor = 0.8        
g2mu = Qs / factor     
g = np.pi * np.sqrt(1 / np.log(Qs / 0.2))          		
mu = g2mu / g**2          	
ir = 0.1 * g**2 * mu         
uv = 10.0       

nevents = 1

### Environment variables

In [2]:
import os

# I need to add this line to ask resources from a specific GPU, which is free. Our GPU server has no queing system
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,4"

os.environ["MY_NUMBA_TARGET"] = "cuda"
# os.environ["MY_NUMBA_TARGET"] = "numba"
os.environ["PRECISION"] = "double"
os.environ['GAUGE_GROUP'] = su_group

# Import relevant modules
import sys
sys.path.append('..')

# Glasma modules
import curraun.core as core
import curraun.mv as mv
import curraun.initial as initial
initial.DEBUG = False

import curraun.su as su
import curraun.lattice as latt
from curraun.numba_target import use_cuda
if use_cuda:
    from numba import cuda

import curraun.su as su
import curraun.coulomb as coulomb

Using CUDA
Using SU(3)
Using double precision


### Simulate glasma

In [3]:
niter = 1
# alpha abelian should be 1/p^2_max 
# p_max = 2*pi/L in lattice units 2*pi/N
alpha = 1/(2*np.pi/N)**2

In [4]:
from numba.core.errors import NumbaPerformanceWarning
import warnings

warnings.simplefilter('ignore', category=NumbaPerformanceWarning)

In [5]:
import pickle
from tqdm import tqdm

# Simulation routine
# def simulate(): 

# Derived parameters
a = L / N
E0 = N / L * hbarc
DT = 1.0 / DTS
maxt = int(tau_sim / a * DTS)

# Initialize Glasma fields
s = core.Simulation(N, DT, g)
va = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
vb = mv.wilson(s, mu=mu / E0, m=ir / E0, uv=uv / E0, num_sheets=ns)
initial.init(s, va, vb)

coul = coulomb.CoulombGaugeTransf(s, alpha)

# with tqdm(total=maxt) as pbar:
# for t in range(maxt):      
#TODO: evolve the glasma fields, at the moment consider only 1 evolution step

# Evolve Glasma fields
core.evolve_leapfrog(s)

coulomb.gauge_transform(coul, alpha)
delta0 = coul.delta0.copy()
c = coul.c.copy()

# print(delta0)
print(c)
                
# pbar.update(1)

if use_cuda:
    cuda.current_context().deallocations.clear()

# return ax

[[ 5.85469173e-18-1.72965896e-01j  7.06283269e-02-1.87717302e-01j
   1.56636718e-01-3.16982275e-02j ... -1.56636718e-01-3.16982275e-02j
   5.38270926e-01+3.91748420e-04j -1.55040911e-17+3.72551780e-01j]
 [ 1.19397844e-17-8.91509938e-02j -5.14399081e-02-1.16977022e-02j
  -3.12944984e-03+1.73413909e-01j ...  3.12944984e-03+1.73413909e-01j
   5.62624765e-01+1.40433216e-01j  4.17722524e-17+3.40209293e-01j]
 [-2.82403114e-17-1.10522303e-01j -1.23936210e-01+1.65353859e-01j
   1.01137890e-01+3.48604675e-01j ... -1.01137890e-01+3.48604675e-01j
   1.25713684e-01+3.93379336e-01j -4.07648427e-17+3.97308797e-01j]
 ...
 [-1.46364645e-17+2.49191458e-01j -2.88404777e-02+1.09414368e-02j
  -2.53836365e-01-2.14233716e-01j ...  2.53836365e-01-2.14233716e-01j
  -9.14388325e-02+1.09161693e-01j -1.61191771e-18-8.20593387e-02j]
 [-1.46495423e-17+3.91690803e-01j -2.68607165e-01+2.75976059e-01j
  -5.56149188e-01-1.82553700e-01j ...  5.56149188e-01-1.82553700e-01j
   6.75697864e-02+1.28318980e-01j  8.62967416e-

### Run simulations

In [6]:
# Supress various horribly long warnings
import warnings
warnings.filterwarnings('ignore')